#### Quick run-through of Part 1 of the amazing langchain tutorial: [fundamentals-part1](https://github.com/gkamradt/langchain-tutorials/tree/main).    

**NOTE**: Global (`RUN_GLOBAL`) and cell-specific (`RUN_LOCAL`) toggles mask the execution of code which makes OpenAI API calls to avoid running out of free OpenAI credits.

### Setup.

In [1]:
# Removed before pushing to GitHub.
openai_api_key='...'
serpapi_api_key='...'

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

RUN_GLOBAL = True

GPT-3.5-Turbo

In [2]:
# Docs show default is gpt-3.5-turbo (probably because it is cost-efficient).
chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    sys_msg = "You are a nice AI bot that helps a user figure out where to travel in one short sentence"
    hum_msg_0 = "I like the beaches where should I go?"
    art_msg = "You should go to Nice, France"
    hum_msg_1 = "What else should I do when I'm there?"
    chat(
        [
            SystemMessage(content=sys_msg),
            HumanMessage(content=hum_msg_0),
            AIMessage(content=art_msg),
            HumanMessage(content=hum_msg_1)
        ]
    )

ADA-001

In [3]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-ada-001", openai_api_key=openai_api_key)

RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    llm("What day comes after Friday?")


DaVinci-003

In [4]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    prompt = """
    Today is Monday, tomorrow is Wednesday.
    What is wrong with that statement?
    """

    llm(prompt)

## Prompt Template

An object that creates prompts based on a combination of user input, other non-static information,. and a fixed template string.

In [5]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)
    
    # Notice "location" below, that is a placeholder for another value later
    template = """
    I really want to travel to {location}. What should I do there?
    
    Respond in one short sentence.
    """
    
    prompt = PromptTemplate(
        input_variables=["location"],
        template=template,
    )
    
    final_prompt = prompt.format(location='Rome')
    
    print (f"Final Prompt: {final_prompt}")
    print ("-----------")
    print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
    I really want to travel to Rome. What should I do there?
    
    Respond in one short sentence.
    
-----------
LLM Output: 
Explore the historical sites, sample the local cuisine, and take in the vibrant culture of the Eternal City.


[Example Selectors](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors)

Select from a series of examples that allow dynamically placing in-context information into the prompt.

In [6]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [7]:
# SemanticSimilarityExampleSelector will select examples that are similar to
# your input by semantic meaning.

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to
    # measure semantic similarity.
    OpenAIEmbeddings(openai_api_key=openai_api_key), 
    
    # This is the VectorStore class that is used to store the embeddings and
    # do a similarity search over.
    FAISS, 
    
    # This is the number of examples to produce.
    k=2
)

In [8]:
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["noun"],
)

In [9]:
# Select a noun!
my_noun = "student"

print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: driver
Example Output: car

Example Input: pilot
Example Output: plane

Input: student
Output:


In [10]:
RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    llm(similar_prompt.format(noun=my_noun))

## Output Parsers

Format the output of a model (usually for structured output).

Two concepts:

1. Format Instructions: An autogenerated prompt tells the LLM how to format its response based off your desired result.
2. Parser: A method which will extract your model's text output into a desired structure (usually json).

In [11]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [12]:
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

In [13]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [14]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [15]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")

print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
welcom to califonya!

YOUR RESPONSE:



In [16]:
RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    llm_output = llm(promptValue)
    llm_output

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

In [17]:
output_parser.parse(llm_output)

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

## Document Loaders

Easy ways to import data from other sources.    
Shared functionality with [OpenAI Plugins](https://openai.com/blog/chatgpt-plugins), specifically [ retrieval plugins](https://github.com/openai/chatgpt-retrieval-plugin).     
See a [big list](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html) of document loaders here.    
A bunch more on [Llama Index](https://llamahub.ai/) as well.

In [18]:
from langchain.document_loaders import HNLoader

In [19]:
loader = HNLoader("https://news.ycombinator.com/item?id=34422627")

In [20]:
data = loader.load()

In [21]:
print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample:

Ozzie_osman 6 months ago  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are very Ozzie_osman 6 months ago  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_index)


Text Splitters: Split long documents into chunks.    
Consider experimenting with [different ones](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html).

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [23]:
# This is a long document we can split up.
with open('./data/PaulGraham-worked.txt') as f:
    pg_work = f.read()
    
print (f"You have {len([pg_work])} document")

You have 1 document


In [24]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 150,
    chunk_overlap  = 20,
)

texts = text_splitter.create_documents([pg_work])

In [25]:
print (f"You have {len(texts)} documents")

You have 610 documents


In [26]:
print ("Preview:")
print (texts[0].page_content, "\n")
print (texts[1].page_content)

Preview:
February 2021Before college the two main things I worked on, outside of school,
were writing and programming. I didn't write essays. I wrote what 

beginning writers were supposed to write then, and probably still
are: short stories. My stories were awful. They had hardly any plot,


## Retrievers

Combine documents with language models.    
Most widely supported: VectoreStoreRetriever.

In [27]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('./data/PaulGraham-worked.txt')
documents = loader.load()

In [28]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    # Embed your texts
    db = FAISS.from_documents(texts, embeddings)

In [29]:
# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()

In [30]:
retriever

VectorStoreRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7f9dbff01290>, search_type='similarity', search_kwargs={})

In [31]:
docs = retriever.get_relevant_documents("what types of things did the author want to build?")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

In [32]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

standards; what was the point? No one else wanted one either, so
off they went. That was what happened to systems work.I wanted not just to build things, but to build things that would
last.In this di

much of it in grad school.Computer Science is an uneasy alliance between two halves, theory
and systems. The theory people prove things, and the systems people
build things. I wanted to build things. 


## VectorStores

Databases to store vectors.

[Chroma](https://www.trychroma.com/) & [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) are easy to work with locally.    
Think of them as tables w/ a column for embeddings (vectors) and a column for metadata.

Example

| Embedding                    | Metadata                 |
| -----------                  | -----------              |
| [-0.0156, -0.0032, ...]      | {'date' : 1/2/23}        |
| [-0.0354, 1.4654, ...]       | {'date' : 1/3/23}        |

In [33]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('./data/PaulGraham-worked.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    # Get embedding engine ready
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [34]:
print (f"You have {len(texts)} documents")

You have 78 documents


In [35]:
embedding_list = embeddings.embed_documents([text.page_content for text in texts])

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

In [36]:
print (f"You have {len(embedding_list)} embeddings")
print (f"Here's a sample of one: {embedding_list[0][:3]}...")

You have 78 embeddings
Here's a sample of one: [-0.0010875738459471215, -0.011166318559573047, -0.012805657736331182]...


## Memory

Help LLMs remember information.

Refer to [the documentation](https://python.langchain.com/docs/modules/memory/) for different types of memory.

ChatMessageHistory is (one of) the most common way to maintain state.

In [37]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI


RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of france?")

In [38]:
history.messages

[AIMessage(content='hi!', additional_kwargs={}, example=False),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, example=False)]

In [39]:
ai_response = chat(history.messages)
ai_response

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False)

In [40]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, example=False),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, example=False),
 AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False)]

## Chains

Combining different LLM calls and action automatically.    
Ex: Summary #1, Summary #2, Summary #3 > Final Summary

See [the chains documentation](https://python.langchain.com/docs/modules/chains/) for a more comprehensive overview.

Type 1/n: Simple Sequential Chains

Easy chains where you can use the output of an LLM as an input into another. Good for breaking up tasks (and keeping your LLM focused)

In [41]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

RUN_LOCAL = False
if RUN_LOCAL or RUN_GLOBAL:
    llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

In [42]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [43]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [44]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

In [45]:
review = overall_chain.run("Rome")



> Entering new SimpleSequentialChain chain...
Pasta alla Carbonara - an iconic Roman dish of pasta with a creamy sauce made of egg, bacon, cheese, and black pepper.

Ingredients:

  - 8 oz. of your favorite pasta
  - 4 strips of bacon (chopped into small pieces)
  - 2 cloves of garlic (minced)
  - 2 eggs
  - ¼ cup of Parmesan cheese (grated)
  - 2 tablespoons of olive oil
  - Salt & pepper to taste

Instructions:

1. Bring a pot of salted water to a boil, add the pasta, and cook until al dente.

2. In a pan over medium-low heat, cook the bacon and garlic for 8 minutes until lightly golden.

3. In a bowl, whisk together the eggs and Parmesan cheese until combined.

4. Once the pasta is cooked, drain and set aside.

5. Add the olive oil to the bacon and garlic in the pan and stir to combine.

6. Increase the heat to medium-high and add the cooked pasta to the pan.

7. Pour the egg and cheese mixture into the pan. Stir continuously until the sauce is smooth and creamy.

8. Season with s

Type 2/n: Summarization Chain

Easily run through long numerous documents and get a summary.

In [46]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('./data/PaulGraham-worked.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Added to get proof of concept more quickly. 113 --> 5
texts = texts[:5]

chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"February 2021Before college the two main things I worked on, outside of school,
were writing and programming. I didn't write essays. I wrote what
beginning writers were supposed to write then, and probably still
are: short stories. My stories were awful. They had hardly any plot,
just characters with strong feelings, which I imagined made them
deep.The first programs I tried writing were on the IBM 1401 that our
school district used for what was then called "data processing."
This was in 9th grade, so I was 13 or 14. The school district's
1401 happened to be in the basement of our junior high school, and
my friend Rich Draves and I got permission to use it. It was like"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"a mini Bond villain's lair down there, with all these alien-looking
machines 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..



> Finished chain.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..




> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


" In February 2021 before college, two of the activities the author worked on were writing and programming. They wrote mostly short stories at the time, and their first programs were written on an IBM 1401 computer at their junior high school with the permission of their friend, Rich Draves.

 A young speaker was visiting a mini Bond villain's lair filled with alien-looking machines where they were using an early version of Fortran. The programs required input data stored on punched cards, but the speaker was puzzled and unsure of what could be done with the 1401.

 This person recounts their earliest experience of programming, which was uneventful due to lack of data stored on punched cards and not knowing enough math to make interesting calculations. They do, however, remember the realization that their programs could not terminate, and the disapproval from the data center manager.

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco


> Finished chain.

> Finished chain.


' The author reflects on their introduction to programming in junior high on an IBM 1401 computer with no data and no real understanding of what they were doing. This led to experimenting with a Heathkit-built microcomputer in the 80\'s and further experimenting with a TRS-80 computer. With microcomputers, a new realm of possibility was created and the author was able to create their own programs and games, including a rocket predictor and a basic word processor. At first, the author saw more promise in philosophy in college, but quickly realized that college involved more than just the study of "ultimate truths".'

## Agents

> Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an **unknown chain** that depends on the user's input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then **decide which, if any, of these tools to call**.


**Agents**: The language model that drives decision making. An agent takes in an input and returns a response corresponding to an action to take along with an action input. See the [documentation](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html) for a comprehensive overview.

**Tools**: A 'capability' of an agent. This is an abstraction on top of a function that makes it easy for LLMs (and agents) to interact with it. Ex: Google search. Also, see [OpenAI plugins](https://platform.openai.com/docs/plugins/introduction).

**Toolkit**:  Groups of tools that the agent can select from.

Let's bring them all together:

In [47]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import json

RUN_LOCAL = True
if RUN_LOCAL or RUN_GLOBAL:
    llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [48]:
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=serpapi_api_key)

In [49]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [50]:
response = agent({"input":"what was the first album of the" 
                    "band that Natalie Bergman is a part of?"})

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..




> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I should try to find out what band Natalie Bergman is a part of.
Action: Search
Action Input: "Natalie Bergman band"

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..



Observation: Natalie Bergman is an American singer-songwriter. She is one half of the duo Wild Belle, along with her brother Elliot Bergman. Her debut solo album, Mercy, was released on Third Man Records on May 7, 2021. She is based in Los Angeles.
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I should search for the debut album of Wild Belle.
Action: Search
Action Input: "Wild Belle debut album"
Observation: Isles
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ykIRTu8lBLkl1U8NgQBKUHMk on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer.
Final Answer: Isles is the debut album of Wild Belle, the band that Natalie Bergman is a part of.

> Finished chain.
